<a href="https://colab.research.google.com/github/thieryw/DeepFake-Audio-Detection/blob/main/02_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 2. Treinamento de Modelos de Detecção

Este notebook é responsável pelo treinamento dos modelos de detecção de deepfake utilizando as features extraídas no passo anterior.

### 📋 Etapas:
1.  **Configuração de Ambiente**: Montagem do Google Drive e importação de bibliotecas.
2.  **Configuração do Treinamento**: Definição de hiperparâmetros e arquitetura.
3.  **Execução do Treinamento**: Utilização do `TrainingService` para treinar o modelo.
4.  **Avaliação e Salvamento**: Verificação das métricas e persistência do modelo no Drive.

## 1. Configuração de Ambiente

In [ ]:
import sys
import os
from pathlib import Path

# 1. Montar Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
    print("✅ Google Drive montado com sucesso.")
except ImportError:
    IN_COLAB = False
    print("⚠️ Executando localmente (não no Colab).")

# 2. Configurar Caminhos
if IN_COLAB:
    PROJECT_PATH = Path("/content/drive/MyDrive/TCC")
else:
    PROJECT_PATH = Path(os.getcwd())

# Adicionar projeto ao PYTHONPATH
if str(PROJECT_PATH) not in sys.path:
    sys.path.append(str(PROJECT_PATH))
    print(f"✅ Caminho adicionado ao sistema: {PROJECT_PATH}")

# Verificar estrutura básica
required_dirs = ['app', 'data', 'models']
for d in required_dirs:
    p = PROJECT_PATH / d
    if not p.exists():
        print(f"⚠️ Diretório não encontrado: {p} (será criado se necessário)")
        p.mkdir(parents=True, exist_ok=True)

In [ ]:
# Importações do Projeto
try:
    from app.domain.services.training_service import TrainingService
    from app.core.config.settings import TrainingConfig
    from app.domain.models.architectures.registry import ARCHITECTURE_REGISTRY
    print("✅ Módulos do projeto importados com sucesso.")
except ImportError as e:
    print(f"❌ Erro ao importar módulos: {e}")
    print("Verifique se o caminho do projeto está correto e se os arquivos __init__.py existem.")

## 2. Configuração do Treinamento

Defina os parâmetros para o treinamento do modelo.

In [ ]:
# --- Parâmetros de Entrada ---

# 1. Arquivo de Dados (gerado no passo anterior)
DATASET_FILENAME = "dataset_features.npz" # @param {type:"string"}
DATASET_PATH = PROJECT_PATH / "data" / "processed" / DATASET_FILENAME

# 2. Arquitetura do Modelo
# Opções disponíveis: AASIST, RawGAT-ST, Light-CNN
ARCHITECTURE = "AASIST" # @param ["AASIST", "RawGAT-ST", "Light-CNN"]

# 3. Hiperparâmetros
BATCH_SIZE = 32 # @param {type:"integer"}
EPOCHS = 10 # @param {type:"integer"}
LEARNING_RATE = 0.0001 # @param {type:"number"}

# 4. Nome do Modelo de Saída (opcional, deixe vazio para auto-gerar)
MODEL_NAME = "" # @param {type:"string"}

print(f"Dataset: {DATASET_PATH}")
print(f"Arquitetura: {ARCHITECTURE}")
print(f"Configuração: Epochs={EPOCHS}, Batch={BATCH_SIZE}, LR={LEARNING_RATE}")

## 3. Execução do Treinamento

In [ ]:
if not DATASET_PATH.exists():
    print(f"❌ Erro: Arquivo de dataset não encontrado em {DATASET_PATH}")
    print("Por favor, execute o notebook '01_Setup_and_Extraction.ipynb' primeiro.")
else:
    # Inicializar Serviço
    models_dir = PROJECT_PATH / "models"
    trainer_service = TrainingService(models_dir=str(models_dir))

    # Preparar Configuração
    training_config = {
        "batch_size": BATCH_SIZE,
        "epochs": EPOCHS,
        "learning_rate": LEARNING_RATE,
        "parameters": {}  # Parâmetros específicos da arquitetura podem ir aqui
    }

    if MODEL_NAME:
        training_config["model_name"] = MODEL_NAME

    print("🚀 Iniciando treinamento... (isso pode levar alguns minutos)")
    
    # Executar Treinamento
    result = trainer_service.train_model(
        architecture=ARCHITECTURE,
        dataset_path=str(DATASET_PATH),
        config=training_config
    )

    # Verificar Resultado
    if result.status.name == "SUCCESS":
        print("\n✅ Treinamento concluído com SUCESSO!")
        metadata = result.data
        print(f"Modelo salvo em: {metadata.file_path}")
        print("\nMétricas:")
        for k, v in metadata.metrics.items():
            print(f"  - {k}: {v}")
    else:
        print("\n❌ Falha no treinamento:")
        for error in result.errors:
            print(f"  - {error}")

## 4. Próximos Passos

Com o modelo treinado, você pode:
1.  **Avaliar o modelo** com mais detalhes (matriz de confusão, curvas ROC).
2.  **Executar inferência** em novos arquivos de áudio.
3.  **Lançar a interface gráfica** para demonstração.

Vá para o próximo notebook: `03_Inference_and_Demo.ipynb`.